# 1505022 [ML Offline 1]

## Import statements

In [290]:
import pandas as pd
import numpy as np
import math
import random
import sklearn
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import datetime

### Load datasets

In [23]:
file_name_dataset_1 = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv"
file_name_dataset_2_train = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/adult.data"
file_name_dataset_2_test = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/adult.test"
file_name_dataset_3 = "F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/creditcardfraud/creditcard.csv"



In [171]:
file_name_dataset = file_name_dataset_2_train
# file_name_dataset = file_name_dataset_3
print(file_name_dataset)
### NEED TO REPLACE THIS IN FILES as HEADERS [adult.data and adult.test]
column_names_dataset_2 = ["age", "workclass", "fnlwgt", "education",
                   "education-num", "marital-status", "occupation",
                   "relationship", "race", "sex", "capital-gain",
                   "capital-loss", "hours-per-week", "native-country", "Label"]
for col in column_names_dataset_2:
    print(col, end = ',')

F:/Programs C and Java/Sessional Things/Assignments-Github/ML Assignments/adult.data
age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Label,

In [292]:
data_frame_original = pd.read_csv(file_name_dataset) 
data_frame_original.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,Label
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [165]:
data_frame_original.shape

(32561, 15)

### Entropy calculation

In [43]:
"""
    Input: examples [numpy array, WITHOUT the labels], labels/classes [numpy array format]
    Output: Entropy of THIS node
    *** epsilon_small is used for log_2 operations (log2(0) may give unwanted exceptions)
    Calculation: 
        for each label x of Labels:
            probability[x] = x/num_examples
        Entropy(node) = H(node) = - [ sum of probability[x]*log_2(probability[x]) ]
"""
def calculate_entropy(examples, labels, epsilon_small = 0.0000000000000000001): # WORKING
    labels_unique = np.unique(labels) # obtain the unique labels of the data
    # gives unique label_names, and counts for each unique label_names
    label_names, label_counts = np.unique(labels,
                                         return_counts = True) 
    label_probabilities = label_counts/sum(label_counts)
    label_log_probabilities = np.log2((label_probabilities + epsilon_small))
    label_products = label_probabilities * label_log_probabilities
#     print(len(examples), " , ", label_names , " , " , label_probabilities, " , ", label_log_probabilities)
#     print(label_products)
    entropy = -1 * sum(label_products)
    if entropy == 0.0:  # to not return -0.0
        entropy = 0.0
    return entropy

In [44]:
"""
    Finds the feature types ...
"""
# If num of unique vals are greater than 20, declare as continuous [if float/int]
def check_data_type_one(unique_vals, threshold_cnt):  
    if len(unique_vals) == 0:
        return "CATEGORICAL"  # just checking
    sample_val = unique_vals[0]
    if isinstance(sample_val, str):
        return "CATEGORICAL"
    if isinstance(sample_val, float):
        return "CONTINUOUS"
    if len(unique_vals) > threshold_cnt:
        return "CONTINUOUS"
    else:
        return "CATEGORICAL"
# If num of unique vals are greater than 20, declare as continuous [if float/int]
def find_data_types(data_frame, threshold=20):
    data_type_list = []
    for feature_test in data_frame.columns.values:
        val_first = data_frame[feature_test]
#         print(feature_test, " : ", val_first, "  :  ", type(val_first))
        unique_vals = np.unique(data_frame[feature_test])
#         print(unique_vals)
        type_val = check_data_type_one(unique_vals, threshold)
        data_type_list.append(type_val)
        # Keep looping
        
    return data_type_list

### Information Gain Calculation

###### Information Gain Calculation wrt one feature_column

In [45]:
"""
    Input: X (examples), Y (labels), feature_column(idx on X), feature_type [CONTINUOUS/CATEGORICAL],
            use_custom_columns {optional}, custom_columns_list {optional/=}
    Output: Information Gain wrt that feature [CATEGORICAL/BINARIZED]
            Dictionary of information gains wrt each values of that feature [CONTINUOUS]
    Dependency: Uses calculate_entropy() function written above
    Calculation:
        for each value v of examples[feature_column]:
            Partition new_examples[v] by choosing that feature.val == v [if categorical]
            Partition new_examples[v] by choosing that feature.val <= v [if continuous]
            calculate entropy of new_examples, H(S_feature_val_v)
            calculate num_examples(S_feature_val_v)/num_examples(parent_node)
            Use formula IG = H(S{parent}) - [Sum of |S_val|/|S| * H(S_val)]
    Treat either as continuous, or categorical [binarized data is treated as categorical]
"""
def calculate_information_gain(X, Y, feature_column, feature_type,
                              use_custom_columns = False, custom_columns_list = None):  # WORKING
    entropy_parent_node = calculate_entropy(X, Y) # entropy of parent node
    # Either use ALL values for this column OR use custom values only
    if ((use_custom_columns == True)) :
        unique_vals_features = custom_columns_list
    else:
        unique_vals_features = np.unique(X[:, feature_column])
#         print("In calculate_info_gain() ... feature_col = ", feature_column, " , printing unique values of the feature : ", unique_vals_features)
#         print("-->>DOING FOR ALL UNIQUE CONTINUOUS COLUMNS, feature-col = ", feature_column,
#             " , feature_type = ", feature_type, " len_unique_feature_vals = ", len(unique_vals_features))
    if feature_type == "CONTINUOUS":
#         print("-->>FEATURE_COL = ", feature_column, " .. inside if feature type == CONTINUOUS")
        # Partition into two sets ... x <= val and x > val
        info_gain_dict = {}
        for val in unique_vals_features:
            idx_left_bool = X[:, feature_column] < float(val)
            idx_right_bool = X[:, feature_column] >= float(val)
            data_left = X[idx_left_bool]
            label_left = Y[idx_left_bool]
            data_right = X[idx_right_bool]
            label_right = Y[idx_right_bool]
            entropy_left = calculate_entropy(data_left, label_left)
            entropy_right = calculate_entropy(data_right, label_right)
            info_gain = entropy_parent_node - (
                ((len(data_left)/len(X)) * entropy_left) + 
                ((len(data_right)/len(X)) * entropy_right)
            )
            info_gain_dict[val] = info_gain
#         print("+++-->>Inside calculate_info_gain() [CONTINUOUS] ... dictionary-len = ", 
#               len(info_gain_dict), " ... printing info_gain_dict .. ", info_gain_dict)
        return info_gain_dict
    else:# CATEGORICAL
        ## Partition into FOR EACH FEATURE
#         entropy_per_val = {} # empty dictionary
        num_examples_parent = len(X)
        cumulative_entropy = 0.0 # cumulative entropy for all features
        if num_examples_parent == 0: # SOMEHOW comes down to this
            print("-->>Inside calculateInfoGain() .. num_examples_parent = ", num_examples_parent,
                 " returning 0")
            return 0
        for val in unique_vals_features:
            idx_equal_to_feature = X[:, feature_column] == val
            data_of_feature = X[idx_equal_to_feature]
            label_of_feature = Y[idx_equal_to_feature]
            entropy_of_feature = calculate_entropy(data_of_feature, label_of_feature)
#             print("val = ", val, " , entropy of feature = ", entropy_of_feature)
            proportion_of_examples_in_feature = float(len(data_of_feature)) / float(num_examples_parent)
#             print("proportion of examples in feature = ", proportion_of_examples_in_feature)
            cumulative_entropy = cumulative_entropy + (proportion_of_examples_in_feature * entropy_of_feature)
#             print("cumulative entropy = ", cumulative_entropy, " parent entropy = ", entropy_parent_node)
        #             entropy_per_val[val] = entropy_of_feature
        # now subtract from parent's entropy to return the information gain
        info_gain = entropy_parent_node - cumulative_entropy
        return info_gain

## Preprocessing datasets
#### Make the last column as 'Label' and rename it to 'Label'
#### Make any string/object datatype to integers [encoding]

In [466]:
"""
Dataset 1: churn dataset
1. Tenure is continuous
2. MonthlyCharges is continuous
3. TotalCharges is continuous [object type ... changed to float type] [Some missing values ... spaces, delete those rows]
4. Drop customerID column
"""
def preprocess_dataset_1(df):    
    df_copy = df.copy(deep = True)
    df_copy.rename(columns = {'Churn' : 'Label'}, inplace=True)
    df_copy = df_copy.drop("customerID", axis = 1)  # drop customer ID
    df_copy.drop(df_copy[df_copy.TotalCharges == ' '].index, inplace=True)  # delete rows with spaces
    df_copy["TotalCharges"] = df_copy["TotalCharges"].astype(float)
    return df_copy

In [262]:
"""
Dataset 2: Adult dataset
Problems: Rows contain '?', Continuous values for age, fnlwgt, education-num, capital-gain, capital-loss, hours-per-week
Columns that have '?' marks are : workclass, occupation , native-country [we replace with MODE]
Col =  workclass , Mode =   Private , Percent =  69.70301894904948  %
Col =  occupation , Mode =   Prof-specialty  , Percent =  12.714597217530175  %
Col =  native-country , Mode =   United-States  , Percent =  89.5857006848684  %
"""
def preprocess_dataset_2_train(df_original):
    df = df_original.copy(deep = True)
    print("No. of columns = ", len(df.columns.values))
    df['education-num'] = df['education-num'].astype(float)  # to make THIS column float [so that it automatically becomes continuous]
    list_columns_with_QUESTION_mark = []
    for col in df.columns.values:
        unique_vals, unique_counts = np.unique(df[col].values, return_counts=True)
        missing_val = " ?"
        if missing_val in unique_vals:  # ONLY THOSE COLUMNS THAT CONTAIN THE ' ?' mark
            list_columns_with_QUESTION_mark.append(col)
#             idx_search = -1  # index of '?' mark
            mode_of_unique_vals = unique_vals[np.argmax(unique_counts)]
#             print("Col = ", col, " Mode = ", mode_of_unique_vals, " Percent = ", (unique_counts[np.argmax(unique_counts)]/len(df))*100, " %")
            df[col] = np.where((df[col] == ' ?'),mode_of_unique_vals,df[col])        
    return df, list_columns_with_QUESTION_mark

data_frame, list_col_with_qstn_mark = preprocess_dataset_2_train(data_frame_original)
data_frame_test_original = pd.read_csv(file_name_dataset_2_test)
data_frame_test, list_col_with_qstn_mark_test = preprocess_dataset_2_train(data_frame_test_original)
X_train = (data_frame.drop("Label", axis = 1)).values
Y_train = (data_frame["Label"]).values
X_test = (data_frame_test.drop("Label", axis = 1)).values
Y_test = (data_frame_test["Label"]).values
print("Train: ", X_train.shape, " ", Y_train.shape, "  Test: ", X_test.shape, " ", Y_test.shape)
# for col in list_col_with_qstn_mark:
#     print("\nCol = ", col, " ", np.unique(data_frame[col].values, return_counts=True))
# for col in list_col_with_qstn_mark_test:
#     print("\nCol = ", col, " ", np.unique(data_frame_test[col].values,return_counts=True))

No. of columns =  15


<ipython-input-262-746da32ce874>:17: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if missing_val in unique_vals:  # ONLY THOSE COLUMNS THAT CONTAIN THE ' ?' mark


No. of columns =  15
Train:  (32561, 14)   (32561,)   Test:  (16281, 14)   (16281,)


In [194]:
"""
Dataset 3: Credit-Card
All are continuous values, so we will binarize them
Drop the 'time' column, keep only 20,000 NO/False labels and keep ALL YES/True labels
To save time, try to binarize using 100 data points
"""
def preprocess_dataset_3(df, data_size = 20000):
    df.rename(columns={'Class':'Label'}, inplace=True)  
    # drop 'Time' column
    df = df.drop("Time", axis = 1)
    
    df_yes = df[df['Label'] == 1]
    df_no  = df[df['Label'] == 0]
    
    indices = df_no.index.tolist()
    test_indices = random.sample(population=indices, k=data_size)  # only keeps 'k' amount of data
    df_no_kept = df.loc[test_indices]
#     df_no_dropped = df.drop(test_indices)
    # recombine the 'YES' and 'NO' samples together into a new dataframe
    df = pd.concat([df_yes, df_no_kept])
    return df

###### Make the 'Label' column as separate Labels and use other columns

In [47]:
# data_frame = preprocess_dataset_1(data_frame_original)
# print(data_frame.head(5))

# data_frame = preprocess_dataset_3(data_frame_original)
# print(data_frame.head(5))

data_frame = preprocess_dataset_2_train(data_frame_original)


In [468]:
"""
    Input: dataframe
    Output: X, Y [numpy format]
"""
def separate_labels_and_features(df):
    X = df.drop("Label", axis = 1)
    Y = df["Label"]
    return X.values, Y.values

In [469]:
X, Y = separate_labels_and_features(data_frame)
print(X.shape)
print(Y.shape)
print(X)

(7032, 19)
(7032,)
[['Female' 0 'Yes' ... 'Electronic check' 29.85 29.85]
 ['Male' 0 'No' ... 'Mailed check' 56.95 1889.5]
 ['Male' 0 'No' ... 'Mailed check' 53.85 108.15]
 ...
 ['Female' 0 'Yes' ... 'Electronic check' 29.6 346.45]
 ['Male' 1 'Yes' ... 'Mailed check' 74.4 306.6]
 ['Male' 0 'No' ... 'Bank transfer (automatic)' 105.65 6844.5]]


In [470]:
print(X.shape, " ", len(X))

(7032, 19)   7032


#### All functions below are to binarize the continuous values of the dataset

In [197]:
"""
    Input: Dataset (X, Y), feature-column
    Output: Best Information Gain wrt THIS feature column
    Dependency: Uses calculate_information_gain(X, Y, feature_column, feature_type) function
                which returns the info_gain [if categorical]
                which returns dictionary of {key = split_val, value = info_gain}
"""
def get_best_IG_val_of_this_feature(X_train, Y_train, feature_col, custom_col_list=None,
                                   use_custom_col_list = False):
#     print(X_train[:, 0])
    if use_custom_col_list == False:
        dict_info_gain_col = calculate_information_gain(X_train, Y_train, 
                  feature_col, feature_type="CONTINUOUS", use_custom_columns=False)
    else:
#         print("Inside using custom columns ... custom_col_list = ", custom_col_list)
        dict_info_gain_col = calculate_information_gain(X_train, Y_train, 
  feature_col, feature_type="CONTINUOUS", use_custom_columns=True, custom_columns_list=custom_col_list)
#     print("-->Inside gt_best_IG_val_of_this_feature() .. col = ", feature_col ,
#           " .. dictionary len = ", len(dict_info_gain_col))
    v = list(dict_info_gain_col.values())
    k = list(dict_info_gain_col.keys())
    return k[v.index(max(v))], max(v) # returns the max gain and index/split_val of that max IG

In [233]:
"""
    To binarize the data, instead of comparing values for EACH continuous value,
    we will divide into 100 (or user defined number of) data points, 
    and compare with each of those values. [To make it time-efficient (Takes ~ 2mins)]
"""
def get_dictionary_of_best_split_values_for_each_col(X, Y, num_values_of_custom_cols = 100,
                        use_custom_col_list = False, custom_cols = None, use_custom_end_points = False,
                                                    which_cols_to_use_endpoints = None):
    _, ncol = X.shape
    num_cols_to_do = np.arange(ncol)
    if use_custom_col_list == True:
        num_cols_to_do = custom_cols
    dict_best_ig_feature_split_values = {}  # Dictionary to store {feature_col: split_val, max_IG} that has max IG wrt that feature column
#     for col_feature in num_cols_to_do:  # EITHER ONLY DO FOR SELECTED COLUMNS/ OR FOR ALL COLUMNS
    for idx_col in range(len(num_cols_to_do)):
        col_feature = num_cols_to_do[idx_col]
        unique_vals_of_this_feature = np.unique(X[:, col_feature])
        left_range = int(np.ceil(min(unique_vals_of_this_feature)))
        right_range = int(np.floor(max(unique_vals_of_this_feature))) 
        cols_custom = np.linspace(left_range, right_range, num_values_of_custom_cols)
        if use_custom_end_points == True:
            if which_cols_to_use_endpoints is not None:
                split_val_for_max_IG, max_IG = get_best_IG_val_of_this_feature(X, Y, col_feature, 
                                                       cols_custom, use_custom_col_list=True)
            else:
                if which_cols_to_use_endpoints[idx_col] == True:
                    split_val_for_max_IG, max_IG = get_best_IG_val_of_this_feature(X, Y, col_feature, 
                               cols_custom, use_custom_col_list=True)
        else:
            # Use ALL values for continuous ....
            split_val_for_max_IG, max_IG = get_best_IG_val_of_this_feature(X, Y, col_feature)
        dict_best_ig_feature_split_values[col_feature] = split_val_for_max_IG, max_IG
        print("Dictionary: Done for Column = ", col_feature, " split-val = ", split_val_for_max_IG, " Max IG = ", max_IG)
    return dict_best_ig_feature_split_values

###### Binarizes the dataset per column wrt one feature_column

In [234]:
"""
    Actually binarizes the Data wrt the above dictionary found.
"""
def binarize_dataset(X, dict_split_values_and_max_IG_per_col):
#     v = list(dict_split_values_and_max_IG_per_col.values())
    X_bin = X
    keys_list = list(dict_split_values_and_max_IG_per_col.keys())
    features_column = {}
#     print(keys_list)
    for col in keys_list:
        # dictionary[col][0] gives split-value and dictionary[col][1] gives max ig
        split_value_of_col = dict_split_values_and_max_IG_per_col[col][0]
        binarized_data_this_col = X[:, col] < split_value_of_col
        X_bin[:, col] = binarized_data_this_col
        features_column[col] = "BINARIZED"
    
    return X_bin, features_column

In [200]:
def get_new_features_list(features_list_binarized, features_list_previous_arr):
    features_list_new = {}
    for itr in range(len(features_list_previous_arr)):
        features_list_new[itr] = features_list_previous_arr[itr]
    for key in list(features_list_binarized.keys()):
        features_list_new[key] = features_list_binarized[key]
    return list(features_list_new.values()) # returns as list

In [203]:
"""
    Label encoding of labels/classes/Y ...
"""
def label_encode_labels(Y):
    le = preprocessing.LabelEncoder()
    le.fit(Y)
    Y= le.transform(Y)
    return Y 

In [204]:
"""
    Label encoding of examples/X .... 
"""
def label_encode_data(X, feature_types):
    le = preprocessing.LabelEncoder()
    for i in range(len(feature_types)):
        if feature_types[i] == "CATEGORICAL":
            le.fit(X[:, i])
            X[:, i] = le.transform(X[:, i])
    return X

In [480]:
df_2 = data_frame.drop(data_frame.columns.values[-1], axis=1)
# print(df_2.head(2))
feature_types_before = find_data_types(df_2)
print(feature_types_before, "\n", len(feature_types_before))
indices_continous = []
for i in range(len(feature_types_before)):  ## ONLY BINARIZE THE CONTINUOUS FEATURES ... [float/num unique values > 15]
    if feature_types_before[i] == "CONTINUOUS":
        indices_continous.append(i)
print(indices_continous)
X, Y = separate_labels_and_features(data_frame)
print("================ BEFORE ======================")
# dict_test = get_dictionary_of_best_split_values_for_each_col(X, Y, 100,
#                                         use_custom_col_list=False)  # FOR ALL SPLIT-VALUES CONSIDERATION
### CONSIDERS ONLY 100 data points from left-most, to right-most and splits accordingly 
## [not much worse wrt dataset 3 accuracy, however a strong improvement in time]
## This method takes ~5s, whereas all unique values of one column takes around 1hr, 20mins [TOO MUCH !!]
dict_test = get_dictionary_of_best_split_values_for_each_col(X, Y, 100,
                                        use_custom_col_list=True, custom_cols=indices_continous)

print(dict_test)
print("-------------------- AFTER -------------------------")
X_binarized, features_col_new = binarize_dataset(X, dict_test)
features_col_new = get_new_features_list(features_col_new, feature_types_before)
print(features_col_new)
X_binarized = label_encode_data(X_binarized, features_col_new)
Y = label_encode_labels(Y)
# print(X_binarized)
# Pass this binarized data into the engine

['CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CONTINUOUS', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CONTINUOUS', 'CONTINUOUS'] 
 19
[4, 17, 18]
================ BEFORE ======================
{4: (17.494949494949495, 0.07579675305132505), 17: (27.0, 0.03823985984108347), 18: (369.1010101010101, 0.0340171106893129)}
-------------------- AFTER -------------------------
['CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'BINARIZED']


In [293]:
"""
Dataset 2:
[Col, #Unique vals, #All vals]
[ 0 ,  73 ,  32561 ] [ 2 ,  21648 ,  32561 ] [ 4 ,  16 ,  32561 ] 
[ 10 ,  119 ,  32561 ] [ 11 ,  92 ,  32561 ] [ 12 ,  94 ,  32561 ]
col=2 NEEDS to have custom data points try, otherwise will take too much time !!
"""
df_2 = data_frame.drop(data_frame.columns.values[-1], axis=1)
feature_types_before = find_data_types(df_2)
print(feature_types_before, "\n", len(feature_types_before))
indices_continous = []
for i in range(len(feature_types_before)):  ## ONLY BINARIZE THE CONTINUOUS FEATURES ... [float/num unique values > 15]
    print(df_2.columns.values[i], " ", feature_types_before[i])
    if feature_types_before[i] == "CONTINUOUS":
        indices_continous.append(i)
print(indices_continous)
print("================ BEFORE ======================")
indices_continous.remove(2) # REMOVE "fnwlgt" column [will discretize it later]
flag_will_use_custom = [False, True, False, False, False, False]
dict_binarized_values = get_dictionary_of_best_split_values_for_each_col(X, Y, 100,
                            use_custom_col_list=True, custom_cols=indices_continous,
                            use_custom_end_points=False)

print(dict_binarized_values)
print("-------------------- AFTER -------------------------")
X_binarized, features_col_new = binarize_dataset(X_train, dict_binarized_values) # binarize
features_col_new = get_new_features_list(features_col_new, feature_types_before)
X_train = label_encode_data(X_binarized, features_col_new) # label encoding [EXAMPLES, Training data]
enc = sklearn.preprocessing.KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans') # 10 bins
# strategies: quantile, kmeans, uniform [encod = 'ordinal' FOR integer]
# X_train[:, 2] = (enc.fit_transform(X_train[:, 2].reshape(-1, 1))).reshape(-1, )
Y_train = label_encode_labels(Y_train) # label encoding [LABELS, Training data]
X_test, features_col_new_test = binarize_dataset(X_test, dict_binarized_values) # binarize
X_test = label_encode_data(X_test, features_col_new) # label encoding [EXAMPLES, TEST DATA]
# X_test[:, 2] = (enc.fit_transform(X_test[:, 2].reshape(-1, 1))).reshape(-1, )  # DISCRETIZATION
Y_test = label_encode_labels(Y_test) # label encoding [LABELS, TEST DATA]
print(features_col_new)

['CONTINUOUS', 'CATEGORICAL', 'CONTINUOUS', 'CATEGORICAL', 'CONTINUOUS', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CONTINUOUS', 'CONTINUOUS', 'CONTINUOUS', 'CATEGORICAL'] 
 14
age   CONTINUOUS
workclass   CATEGORICAL
fnlwgt   CONTINUOUS
education   CATEGORICAL
education-num   CONTINUOUS
marital-status   CATEGORICAL
occupation   CATEGORICAL
relationship   CATEGORICAL
race   CATEGORICAL
sex   CATEGORICAL
capital-gain   CONTINUOUS
capital-loss   CONTINUOUS
hours-per-week   CONTINUOUS
native-country   CATEGORICAL
[0, 2, 4, 10, 11, 12]
================ BEFORE ======================
Dictionary: Done for Column =  0  split-val =  28  Max IG =  0.0737162839458736
Dictionary: Done for Column =  4  split-val =  13.0  Max IG =  0.07069368427626688
Dictionary: Done for Column =  10  split-val =  7298  Max IG =  0.08695927355015609
Dictionary: Done for Column =  11  split-val =  1825  Max IG =  0.023190335847712906
Dictionary: Done for Column =  12  split-val =  42

In [264]:
# # Column 2 issue ..
# col_2_unique_vals = X_train[:, 2]
# print(col_2_unique_vals.shape)
# print(col_2_unique_vals)
# col_2_unique_vals = col_2_unique_vals.reshape(-1, 1)
# # sklearn.preprocessing.KBinsDiscretizer(n_bins=5, encode='onehot', strategy='quantile')

# enc = sklearn.preprocessing.KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='kmeans')
# col_2_binned = enc.fit_transform(col_2_unique_vals)
# print(col_2_unique_vals)
# print(col_2_binned, " ", col_2_binned.shape, np.unique(col_2_binned))
# col_2_binned = col_2_binned.reshape(-1, )
# print(col_2_binned, " ", col_2_binned.shape, np.unique(col_2_binned))

In [483]:
"""
    Compute new feature types ... [Binarized/Categorical/Continuous]
"""
def obtain_new_feature_types(feature_types_prev, dict_feature_types):  # WORKING
    keys = list(dict_feature_types.keys())
    feature_types_latest = feature_types_prev # Copy previous feature types ...
    for i in keys:
        feature_types_latest[i] = dict_feature_types[i]
    return feature_types_latest

In [269]:
# feature_types = obtain_new_feature_types(feature_types_before, features_col_new)
# print(feature_types, " ", len(feature_types))
feature_types = features_col_new
print(feature_types)

['BINARIZED', 'CATEGORICAL', 'CONTINUOUS', 'CATEGORICAL', 'BINARIZED', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'CATEGORICAL', 'BINARIZED', 'BINARIZED', 'BINARIZED', 'CATEGORICAL']


### Convert to numpy and train_test_split using scikitlearn

In [485]:
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.metrics import confusion_matrix

In [486]:
X_train, X_test, Y_train, Y_test = train_test_split(X_binarized, Y, test_size=0.2, random_state=8)
# X_train, X_test, Y_train, Y_test = train_test_split(X_binarized, Y, test_size=0.2)

In [693]:
print(X_train.shape, " ", Y_train.shape , " ", X_test.shape, " ", Y_test.shape)

(5625, 19)   (5625,)   (1407, 19)   (1407,)


### Form decision tree bulding algorithm

In [270]:
"""
    Returns the majority of the label
"""
def return_majority_label(labels):
    # obtains unique labels AND also the counts of those unique labels
    label_names, label_counts = np.unique(labels, return_counts = True)
    index_max = label_counts.argmax()
    labels_with_max_count = label_names[index_max]
    return labels_with_max_count

In [271]:
"""
    Class DecisionTreeNode to store the decision trees/subtrees nodes.
    CAN'T HAVE CONTINUOUS DATA [ONLY BINARIZED/CATEGORICAL]
"""
class DTreeNode:
    def __init__(self):
        # self.sub_trees = []  # List of subtrees/children
        # self.feature_values = [] # list of feature values of children [To ask question use indices]
        self.children = {} # [feature_val: DTreeNode]
        self.feature_col = -1 # which feature column THIS node contains as a question
        self.is_leaf_node = False # by default, shouldn't be a leaf
        self.classification = "NONE" # also consider as the plurality value
        
    def printTree(self, spaces_num = 0):
        node = self
        if node.is_leaf_node == True:  # Only print the classification column/feature
            print(" " * spaces_num, "Lab(", node.classification, ")")
            return
        else:  # Print the question
            # print(" " * spaces_num, node.feature_col, " ", node.get_comparison_mark(), " ", node.feature_val)
            print("\n", ("    " * spaces_num), "Q(", node.feature_col, ")")
        spaces_num = spaces_num + 1
        for key in list(self.children.keys()):
            print("  "*spaces_num, " == ", key)
            node = self.children[key]
            node.printTree(spaces_num)
            

In [277]:
"""
    Prints various metrics.
"""
def print_metrics(Y_true, Y_predicted):
    # print(len(Y_pred))
    TN, FP, FN, TP = confusion_matrix(Y_true, Y_predicted).ravel()
    accuracy = (TP + TN)/(TP+TN+FP+FN)
    recall = (TP)/(TP + FN)
    specificity = (TN)/(TN + FP)
    precision = (TP)/(TP + FP)
    false_discovery_rate = (FP)/(TP + FP)
    f1_score = 2*((precision * recall) / (precision + recall))
    print("TN = ", TN, " FP = ", FP, " FN = ", FN, " TP = ", TP)
    print("Accuracy = ", accuracy*100, "%")
    print("TPR = Sensitivity = Recall = ", recall*100, "%")
    print("TNR = Specificity = ", specificity*100, "%")
    print("Precision = PPV = Positive Predictive Value = ", precision*100, "%")
    print("FDR = False Discovery Rate = ", false_discovery_rate*100, "%")
    print("F1 Score = ", f1_score*100, "%")

In [283]:
"""
    Can't have CONTINUOUS data [Will be binarized first]
    Decision Tree Classifier
"""    
class DTreeClassifier:
    def __init__(self):
        self.d_tree_root = DTreeNode()
        self.max_depth = 5
        self.print_termination_msg = True
    
    def form_a_leaf_node(self, labels):
        leaf_node = DTreeNode()
        leaf_node.is_leaf_node = True
        leaf_node.classification = return_majority_label(labels)
        return leaf_node
    ##### Can't have CONTINUOUS data [ONLY CATEGORICAL/BINARIZED DATA IS ALLOWED !!]
    def recursive_fit(self, X, Y, X_parent, Y_parent, current_depth, max_depth):
        ## 1. If current-depth == max-depth [Base Case]
        if current_depth == max_depth:
            return self.form_a_leaf_node(Y) # return the leaf node with majority of the current labels
        ## 2. If EXACTLY one label ... return that
        if (len(np.unique(Y)) == 1):
            return self.form_a_leaf_node(Y) # return the leaf node with majority of the current labels
        ## 2_2. If no more examples, return max label of parent
        if len(X) == 0:
            return self.form_a_leaf_node(Y_parent)
        
        current_depth = current_depth + 1  # increment current_depth variable
        val_max_IG = col_max_IG = -1
        for col in range(0, X.shape[1]):  ## for each number of columns/features
            ig_this_col = calculate_information_gain(X, Y, col, "CATEGORICAL")
#             print("col = ", col, " ig_col = ", ig_current_col)
            if ig_this_col > val_max_IG:
                val_max_IG = ig_this_col
                col_max_IG = col
        
        ## 3. Max IG obtained is 0 [no more examples/features]
        if val_max_IG == 0:
            return self.form_a_leaf_node(Y)
        
        d_tree_node = DTreeNode()
        d_tree_node.classification = return_majority_label(Y)  ### Saves the pluarility value of THIS node [used in prediction]
        ## -> Recursion
        for val_of_this_feature in np.unique(X[:, col_max_IG]):
            index_child = (X[:, col_max_IG] == val_of_this_feature)
            X_child = X[index_child]
            Y_child = Y[index_child]

            d_tree_node.is_leaf_node = False  ### is an internal node
            d_tree_node.feature_col = col_max_IG  ### question is to be asked on this column/feature
            # d_tree_node.feature_values.append(val_of_this_feature)
#             if (X_child.shape[0] == 0): # no more examples [DON'T return as other values of list will be empty]
#                 d_tree_node.children.children[val_of_this_feature] = self.form_a_leaf_node(Y)  # return parent's max plurality value
#                 # d_tree_node.sub_trees.append(self.form_a_leaf_node(labels))
#             else:
            ##### DICTIONARY {key = feature_val, value = sub-tree}
            sub_tree = self.recursive_fit(X_child, Y_child, X, Y, current_depth, max_depth)
            d_tree_node.children[val_of_this_feature] = sub_tree
                # d_tree_node.sub_trees.append(sub_tree)
        ### END OF FOR LOOP, return d_tree_node
        return d_tree_node
    
    def printTree(self):
        print("-->Inside printTree")
        self.d_tree_root.printTree()

    def fit(self, examples, labels):
        dt_node = self.recursive_fit(examples, labels, examples, labels,
                                     current_depth = 0, max_depth=self.max_depth)
        self.d_tree_root = dt_node
        if self.print_termination_msg == True:
            print("Fit done for, max-depth = ", self.max_depth)
        
    def predict_one_example(self, example_to_predict):
        self.d_tree_root_backup = self.d_tree_root
        root = self.d_tree_root
        if root == None:
            raise Exception('Root of the Decision Tree is null !! [In predict()]')
        while root is not None:
            if root.is_leaf_node == True: # classify ... since, it is the leaf
                self.d_tree_root = self.d_tree_root_backup
                return root.classification
            else:
                col_to_process = root.feature_col
                val_present_in_example = example_to_predict[col_to_process]
                bool_found = False
#                 print("In col = ", col_to_process, " len root.children = ", len(root.children))
                # for idx in range(len(root.feature_values)):
                keys_feature_vals = list(root.children.keys())
                for feature_val in keys_feature_vals:
                    if example_to_predict[root.feature_col] == feature_val:
                        root = root.children[feature_val]
                        bool_found = True
                        break
                    
                if bool_found == False: ### SHOULD RETURN PARENT's PLURALITY VALUE
                    self.d_tree_root = self.d_tree_root_backup
                    return root.classification # PARENT's plurality value
                    #raise Exception("The value of ", val_present_in_example, " doesn't exist for col_idx = ", col_to_process)
                
    def predict(self, examples_test):
        labels = []
        for example in examples_test:
            yp1 = self.predict_one_example(example)
            labels.append(yp1)
        return labels
        # return TN, FP, FN, TP, accuracy, recall, specificity, precision, false_discovery_rate, f1_score

In [284]:
d_tree_custom_DT = DTreeClassifier()
# d_tree.fit(X_train[0:100], Y_train[0:100], max_depth = 10000)
d_tree_custom_DT.max_depth = 10000 # here, depth-max is not to be considered ...
d_tree_custom_DT.fit(X_train, Y_train)
# 5 s for dataset 1
# 12 s for dataset 2
# 10 s for dataset 3 [with 20k samples]

Fit done for, max-depth =  10000


In [520]:
d_tree_custom_DT.printTree()
# print(np.unique(Y_train, return_counts=True))

-->Inside printTree

  Q( 14 )
    ==  0
  Lab( 0 )
    ==  1
  Lab( 0 )
    ==  2
  Lab( 0 )


### Prediction Script

In [285]:
Y_pred_train = d_tree_custom_DT.predict(X_train)
print("------------- Training ----------------")
print_metrics(Y_train, Y_pred_train)
Y_pred_test = d_tree_custom_DT.predict(X_test)
print("------------- Testing ----------------")
print_metrics(Y_test, Y_pred_test)

------------- Training ----------------
TN =  23995  FP =  725  FN =  1826  TP =  6015
Accuracy =  92.16547403335278 %
TPR = Sensitivity = Recall =  76.71215406198189 %
TNR = Specificity =  97.06715210355988 %
Precision = PPV = Positive Predictive Value =  89.2433234421365 %
FDR = False Discovery Rate =  10.756676557863502 %
F1 Score =  82.50462931211851 %
------------- Testing ----------------
TN =  11359  FP =  1076  FN =  1617  TP =  2229
Accuracy =  83.45924697500153 %
TPR = Sensitivity = Recall =  57.956318252730114 %
TNR = Specificity =  91.3470044229996 %
Precision = PPV = Positive Predictive Value =  67.44326777609683 %
FDR = False Discovery Rate =  32.556732223903175 %
F1 Score =  62.34093133827437 %


In [294]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(criterion='entropy')
dtc.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [295]:
Y_p_train = dtc.predict(X_train)
Y_p_test = dtc.predict(X_test)
print("=============== TRAINING =================")
print_metrics(Y_p_train, Y_train)
print("+++++++++++++++ TESTING ++++++++++++++++++")
print_metrics(Y_p_test, Y_test)

=============== TRAINING =================
TN =  23542  FP =  2466  FN =  1178  TP =  5375
Accuracy =  88.80869752157489 %
TPR = Sensitivity = Recall =  82.02350068670837 %
TNR = Specificity =  90.51830206090433 %
Precision = PPV = Positive Predictive Value =  68.54992985588572 %
FDR = False Discovery Rate =  31.45007014411427 %
F1 Score =  74.68389606780603 %
+++++++++++++++ TESTING ++++++++++++++++++
TN =  11079  FP =  1865  FN =  1356  TP =  1981
Accuracy =  80.2162029359376 %
TPR = Sensitivity = Recall =  59.36469883128559 %
TNR = Specificity =  85.59177997527813 %
Precision = PPV = Positive Predictive Value =  51.508060322412895 %
FDR = False Discovery Rate =  48.491939677587105 %
F1 Score =  55.15801197271336 %


In [288]:
class AdaBoost:
    def __init__(self, mode="DECISION_STUMP"):
        self.mode = mode
    
    # factory method to produce classifier
    def produce_classifier_generic(self): # for now, default is decision stump
        if self.mode == "DECISION_STUMP":
            classifier = DTreeClassifier()
            classifier.print_termination_msg = False
            classifier.max_depth = 1 # decision stump
        else:  # Other modes depending on the 'mode' variable
            classifier = None
        return classifier
    
    # def train_generic(self, data_examples, data_labels):
        # classifier = self.produce_classifier_generic(mode="DECISION_STUMP")
        # classifier.fit(data_examples, data_labels)
        # return classifier
    
    def normalize(self, w):
        if sum(w) == 0:
            return w
        else:
            return w/sum(w)
    
    def resample(self, examples, labels, w):
        # returns resampled data and labels according to weights
        w = self.normalize(w)
        resampled_data_size = len(labels)
        temp_arr = np.arange(resampled_data_size)
        indices_sampled = np.random.choice(temp_arr, len(temp_arr), p=w)  # N resampled data points for now
        examples_sampled = []
        labels_sampled = []
#         print("Inside resample ... indices_sampled = ", indices_sampled)
        for idx in indices_sampled:  # probably a better method exists using numpy
            examples_sampled.append(examples[idx])
            labels_sampled.append(labels[idx])
        
        return np.asarray(examples_sampled), np.asarray(labels_sampled)
    
    # Return the label with the maximum weight of classifier
    def predict(self, example):
        # h: contains the K number of classifiers
        # z: contains the weights of each classifier
        labels_hypothesis = np.zeros(self.K) # Initially set to zeros
        unique_labs_weights = {}
        for k in range(self.K):
            labels_hypothesis[k] = self.h[k].predict_one_example(example)            
        for unique_labels in (np.unique(labels_hypothesis)):
            unique_labs_weights[unique_labels] = 0.0 # initially set as 0 weight
        for k in range(self.K):
            unique_labs_weights[labels_hypothesis[k]] = unique_labs_weights[labels_hypothesis[k]] + self.z[k] 
        
        v = list(unique_labs_weights.values())
        k = list(unique_labs_weights.keys())
        return k[v.index(max(v))]  # return the unique label with the MAX weight
        
        
    def boost(self, examples, labels, K): # K : number of classifiers        
#         L_weak = self.produce_classifier_generic() # generic interface [in this case a decision stump]
        # Local Variables
        self.K = K
        N = examples.shape[0]
        self.w = np.full(shape=(N, ), fill_value = (1/N)) # w, a vector of 'N' weights, 1/N initially
#         h = np.full(shape=(K, ), fill_value = 0) # h, a vector of K hypothesis
#         z = np.full(shape=(K, ), fill_value = 0) # z, a vector of K hypothesis weights
        self.h = {} # dictionary
        self.z = np.zeros(shape=(K, ))
        for k in range(K):
#             print("k = ", k)
            data_resampled, labels_resampled = self.resample(examples, labels, self.w)
            self.h[k] = self.produce_classifier_generic()
            # FIT on RESAMPLED data [labels are also resampled]
            self.h[k].fit(data_resampled, labels_resampled) 
#             h[k] = L_weak
            error = 0
            for j in range(0, N):
                if self.h[k].predict_one_example(examples[j]) != labels[j]: # PREDICT on INITIAL EXAMPLES
                    error = error + self.w[j]
#             print("-->>k = ", k, ", Error = ", error)
            if error > 0.5:
                continue
            for j in range(0, N):
                if self.h[k].predict_one_example(examples[j]) == labels[j]:
                    self.w[j] = self.w[j] * (error/(1 - error))  # error CAN'T be 1 due to continue condition
#             print("+++>> k = ", k, " , w = ", w)
            # Normalize w
            
            self.w = self.normalize(self.w)
            # Error can be 0
            if error == 0:
                self.z[k] = 10 # max of 10^10 ?? [impossible case so, assign a large number]
            else:
                self.z[k] = np.log((1-error)/error)
        
#         return self.Weighted_Majority(h, z)
    

In [291]:
k_list = [5, 10, 15, 20]
import datetime
t_before = datetime.datetime.now()
for K in k_list:
    adaBoost = AdaBoost()
    adaBoost.boost(X_train, Y_train, K)
    Y_pred_train_adaBoost = []
    for ex in X_train:  # predict train dataset
        Y_pred_train_adaBoost.append(adaBoost.predict(ex))
    Y_pred_test_adaBoost = []
    for ex in X_test:  # predict test dataset
        Y_pred_test_adaBoost.append(adaBoost.predict(ex))
    print("------------- Train AdaBoost K = ", K, "-----------------")
    print_metrics(Y_train, Y_pred_train_adaBoost)
    print("------------- Test AdaBoost K = ", K, "-----------------")
    print_metrics(Y_test, Y_pred_test_adaBoost)
    
t_after = datetime.datetime.now()
del_t = t_after - t_before
print("Total time taken = ", del_t.seconds, " s")

------------- Train AdaBoost K =  5 -----------------
TN =  23495  FP =  1225  FN =  4634  TP =  3207
Accuracy =  82.00608089432143 %
TPR = Sensitivity = Recall =  40.90039535773498 %
TNR = Specificity =  95.04449838187702 %
Precision = PPV = Positive Predictive Value =  72.36010830324909 %
FDR = False Discovery Rate =  27.6398916967509 %
F1 Score =  52.2610608653141 %
------------- Test AdaBoost K =  5 -----------------
TN =  11814  FP =  621  FN =  2281  TP =  1565
Accuracy =  82.17554204287207 %
TPR = Sensitivity = Recall =  40.69162766510661 %
TNR = Specificity =  95.00603136308806 %
Precision = PPV = Positive Predictive Value =  71.59194876486734 %
FDR = False Discovery Rate =  28.408051235132664 %
F1 Score =  51.88992042440318 %
------------- Train AdaBoost K =  10 -----------------
TN =  23585  FP =  1135  FN =  4092  TP =  3749
Accuracy =  83.94705322318111 %
TPR = Sensitivity = Recall =  47.81277898227267 %
TNR = Specificity =  95.40857605177993 %
Precision = PPV = Positive Pr